# 情感分析网络应用

_机器学习工程师纳米学位课程 | 部署_

---

在此 notebook 中，我们将使用 Amazon SageMaker 服务构建一个随机树模型来预测影评的情感。此外，我们会将该模型部署到端点上，并构建一个与模型的部署端点交互的简单网络应用。

## 一般步骤

通常，在 notebook 实例中使用 SageMaker 时，你需要完成以下步骤。当然，并非每个项目都要完成每一步。此外，有很多步骤有很大的变化余地，你将在这些课程中发现这一点。

1. 下载或检索数据。
2. 处理/准备数据。
3. 将处理的数据上传到 S3。
4. 训练所选的模型。
5. 测试训练的模型（通常使用批转换作业）。
6. 部署训练的模型。
7. 使用部署的模型。

在此 notebook 中，我们将完成上述每一步。你会发现最后一步（使用部署的模型）很有挑战性。

## 第 1 步：下载数据

我们要使用的数据集很受自然语言处理领域的研究者欢迎，通常称为 [IMDB 数据集](http://ai.stanford.edu/~amaas/data/sentiment/)。其中包含网站 [imdb.com](http://www.imdb.com/) 上的影评，每条影评都标有 '**pos**itive'，表示评论者喜欢影片，否则标有 '**neg**ative'。

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

我们先通过 Jupyter Notebook 功能下载和提取该数据集。

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## 第 2 步：准备和处理数据

我们下载的文件拆分成了各种文件，每个都包含一条影评。我们需要将这些文件合并成两个文件，一个用于训练，一个用于测试。

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## 处理数据

合并并准备好训练和测试数据集后，我们需要将原始数据处理成机器学习算法能够使用的格式。首先，删除所有的 HTML 格式标记以及非字母数字字符。方法很简单，使用 Python 的正则表达式模块即可。稍后我们会讲解为何执行这个很简单的预处理步骤。

In [ ]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [ ]:
review_to_words(train_X[100])

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_web_app")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

### 提取词袋特征

对于我们要实现的模型，我们并不直接使用影评，而是将每条影评转换成词袋特征表示法。注意，我们只能访问训练集，所以转换器只能使用训练集创建表示结果。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size)
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

In [ ]:
len(train_X[100])

## 第 3 步：将数据上传到 S3

创建了训练（和测试）数据的特征表示结果后，我们将开始设置和使用 SageMaker 提供的 XGBoost 分类器。

### 写入数据集

我们将使用的 XGBoost 分类器要求我们将数据集写入文件中并将文件上传到 Amazon S3。我们首先将训练数据集拆分成两部分，分别是训练集和验证集。然后，将这些数据集写入本地文件中，并将文件上传到 S3。此外，我们将测试集写入文件中并将该文件上传到 S3。这样才能使用 SageMaker 批转换功能测试拟合后的模型。

In [ ]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

SageMaker 中的 XGBoost 算法的参考文档要求训练集和验证集不包含标题或索引，并且每个样本的标签在前面。

要详细了解此算法以及其他算法，请参阅 [Amazon SageMaker 开发人员文档](https://docs.aws.amazon.com/sagemaker/latest/dg/)。

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/sentiment_web_app'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

test_X = train_X = val_X = train_y = val_y = None

### 将训练/验证文件上传到 S3

Amazon S3 服务允许我们存储文件，内置训练模型（例如我们将使用的 XGBoost 模型）和自定义模型（例如我们稍后将查看的模型）可以访问这些文件。

对于此任务以及将使用 SageMaker 完成的大多数其他任务，我们可以使用两种方法。一种是使用 SageMaker 的低阶方法，低阶方法要求我们知道在 SageMaker 环境中出现的每个对象。第二种是使用高阶方法，SageMaker 会代替我们做出一些选择。低阶方法的好处是给用户带来了很高的灵活性，而高阶方法使开发速度快多了。对我们来说，我们将使用高阶方法，但是也可以使用低阶方法。

方法 `upload_data()` 是代表当前 SageMaker 会话的对象的成员。该方法会将数据上传到默认存储桶（如果不存在的话，将会创建），并放入由 key_prefix 变量指定的路径下。上传数据文件后，你可以转到 S3 控制台并看看文件上传到哪了。

要查看其他资源，请参阅 [SageMaker API 文档](http://sagemaker.readthedocs.io/en/latest/)以及 __[SageMaker 开发人员指南](https://docs.aws.amazon.com/sagemaker/latest/dg/)__。

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-web-app'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## 第 4 步：创建 XGBoost 模型

上传数据后，下面开始创建 XGBoost 模型。首先需要进行设置。此刻有必要讨论下模型在 SageMaker 中的含义。简单来说，可以将模型看作由 SageMaker 生态系统中的三个不同对象组成，它们相互交互。

- 模型工件
- 训练代码（容器）
- 推理代码（容器）

你可以将模型工件看作实际模型本身。例如，在构建神经网络时，可以将模型工件看作各个层级的权重。在我们的示例中，XGBoost 模型的模型工件是在训练过程中创建的实际树。

训练代码和推理代码然后将用来操纵训练工件。更准确地说，训练代码使用提供的训练数据并创建模型工件，而推理代码使用模型工件对新数据做出预测。

SageMaker 使用 Docker 容器运行训练和推理代码。暂时将容器看作一种代码打包方式，使依赖项不存在问题。

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is required when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# First we create a SageMaker estimator object for our model.
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# And then set the algorithm specific parameters.
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

### 拟合 XGBoost 模型

设置好模型后，我们只需附加训练集和验证集，然后要求 SageMaker 设置计算过程。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

## 第 5 步：测试模型

拟合 XGBoost 模型后，下面看看模型的效果如何。我们将使用 SageMaker 的批转换功能。通过批转换可以轻松地对大型数据集进行推理，因为它并非实时执行。我们并不需要立即使用模型的结果，可以对大量样本进行推理。示例行业应用包括月末报告。这种推理方法的另一个用处是可以对整个测试集进行推理。

为了执行批转换，我们首先需要根据训练过的 estimator 对象创建一个 transformer 对象。

In [ ]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

接下来执行转换作业。我们需要指定要发送的数据的类型，使 SageMaker 能够在后台正确地序列化数据。我们将向模型提供 csv 数据，所以指定为 `text/csv`。此外，如果我们提供的测试数据太大，无法一次性处理完，我们需要指定文件的拆分方式。因为数据集中的每行就是一个条目，所以我们将按照每行拆分输入数据。

In [ ]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

目前转换作业已经在运行，不过是在后台运行。因为我们要等待转换作业运行完毕，所以可以使用 `wait()` 方法查看运行进度。

In [ ]:
xgb_transformer.wait()

现在转换作业已经执行并且结果（每条影评的预测情感）已经保存到 S3 上。因为我们要在本地分析文件，所以通过一个 notebook 功能将文件复制到 `data_dir`。

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

最后一步是读入模型的输出，将输出转换成可用的格式，我们希望情感为 `1`（正面）或 `0`（负面），然后与真实标签进行比较。

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## 第 6 步：部署模型

构建并拟合模型后，SageMaker 将存储生成的模型工件，我们可以使用这些工件部署端点（推理代码）。请打开 SageMaker 控制台，你将看到 SageMaker 创建了一个模型，并且能看到模型工件在 S3 上的存储位置链接。

部署端点与训练模型很相似，不过有几个重要区别。首先是部署的模型不会更改模型工件，所以当你向它发送各种测试样本时，模型不会更改。另一个区别是因为我们没有执行固定计算（训练步骤或批转换时会执行固定计算），所以启动的计算实例会一直运行，直到我们停止它。这一点很重要，因为如果忘记关闭了，将会一直计费。

换句话说，**如果不再使用部署的端点，请关闭！**

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

### （再次）测试模型

部署端点后，我们可以向其发送测试数据并获得推理结果。我们之前已经使用 SageMaker 的批转换功能测试模型，但是这次将使用新部署的端点再次测试模型，以便检查端点是否能正常运行，并了解端点的运行效果。

在使用创建的端点时，需要注意的是，每次调用能够发送的信息量是有限的，所以需要将测试数据分成一份份，然后一份份地发送。此外，我们需要序列化数据，然后才能发送给端点，以确保数据能正确传输。幸运的是，只要提供了数据格式，SageMaker 就能为我们执行序列化步骤。

In [ ]:
from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    
    return np.fromstring(predictions[1:], sep=',')

In [ ]:
test_X = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values

predictions = predict(test_X)
predictions = [round(num) for num in predictions]

最后，看看模型的准确率有多高。

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

结果应该与之前使用批转换作业测试模型后的结果一致。

### 清理数据

确认部署的模型行为符合预期后，我们将关闭端点。注意，端点运行时间越久，费用就越高，因为在我们的简单网络应用能够使用端点之前，还有一些准备工作要做，所以我们将关闭一切。

In [ ]:
xgb_predictor.delete_endpoint()

## 第 7 步：使用模型

我之前多次提到，我们的目标是部署模型并用很简单的网络应用访问该模型。这款网络应用能够接受用户提交的数据（影评），将数据发送给端点（模型），然后显示结果。

但是，还有一个小问题。目前，我们只能使用 SageMaker API 访问端点并向其发送数据。我们可以获得向模型端点发送数据的实际 URL，但是如果我们自己向其发送数据，将不会获得任何结果。这是因为 SageMaker 创建的端点要求访问端点的实体具有相应的权限。所以我们需要向 AWS 验证网络应用的身份。

向 AWS 验证网站的身份似乎超出了这节课的范畴，所以我们将采用另一种方法。我们将创建新的端点，该端点不需要验证身份，并且充当 SageMaker 端点的代理。

还有一个约束条件，我们将避免在网络应用本身里执行任何数据处理步骤。在构建和测试模型时，原始数据是影评，然后我们删除了 HTML 格式标记和标点，并创建了词袋嵌入，然后将生成的向量发送给模型。所有这些处理步骤还要应用到用户输入上。

幸运的是，我们可以使用 Amazon 的 Lambda 服务在后台完成所有这些处理步骤。

<img src="Web App Diagram.svg">

上面的示意图解释了各种服务是如何协同工作的。最右边的是模型，我们在上面训练了模型，并且使用 SageMaker 部署了模型。最左侧的是网络应用，应用将收集用户的影评，将其发送给端点，并获得正面或负面情感预测。

中间的部分比较关键。我们将创建一个 Lambda 函数，你可以将其看做一个简单的 Python 函数，每次发生特定的事件时，该 Python 函数就会被执行。该 Python 函数将对用户提交的输入执行必要的数据处理步骤。此外，该函数有权向 SageMaker 端点发送数据及从中接收数据。

最后，我们执行 Lambda 函数所使用的是新的端点，我们将使用 API Gateway 创建该端点。端点获得数据后，会将数据传递给 Lambda 函数，并返回 Lambda 函数返回的结果。它充当了网络应用与 Lambda 函数之间的通信接口。
### 处理单个影评

假设用户提交了如下所示的字符串影评：

In [ ]:
test_review = "Nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies, totally devoid of any heart or heat. A romantic comedy that has zero romantic chemestry and zero laughs!"

如何将此字符串变成模型要求的词袋特征向量？

在此 notebook 的开头，我们首先使用 `review_to_words` 方法删除所有不需要的字符。我们故意采用了很简单的方式，这是因为我们需要将此方法复制到最终的 Lambda 函数（稍后详细讲解），所以该方法必须很简单。

In [ ]:
test_words = review_to_words(test_review)
print(test_words)

接着，我们需要构建 `test_words` 字符串的词袋嵌入。词袋嵌入使用了 `vocabulary`，其中包含一组文档中最常出现的字词。然后，对于词汇表中的每个字词，我们都记录该字词出现在 `test_words` 中的次数。我们之前使用训练集构建了 `vocabulary`，所以创建 `test_words` 的词袋编码相对比较简单。

In [ ]:
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow

In [ ]:
test_bow = bow_encoding(test_words, vocabulary)
print(test_bow)

In [ ]:
len(test_bow)

现在我们知道如何为提供的影评创建词袋嵌入，如何将其发送给端点呢？首先，我们需要启动端点。

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

此刻，我们可以像之前测试部署的模型并使用 `xgb_predictor` 对象将 `test_bow` 发送给端点一样操作。但是，当我们最终构建 Lambda 函数后，我们将无法访问此对象，所以如何调用 SageMaker 端点？

实际上，Lambda 中使用的 Python 函数能够访问另一个 Amazon 库，叫做 `boto3`。此库提供了使用 Amazon 服务的 API，包括 SageMaker。首先，我们需要获得 SageMaker 运行时的句柄。

In [ ]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

能够访问 SageMaker 运行时后，我们可以要求它使用（调用）已经创建的端点。但是，我们需要告诉 SageMaker 已部署端点的名称。我们可以使用 `xgb_predictor` 对象输出端点名称。

In [ ]:
xgb_predictor.endpoint

我们可以使用运行时调用端点并传入端点名称，然后向其发送 `test_bow` 数据。

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = test_bow)

为何出错了？

因为我们向端点发送了整数列表，但是它要求我们发送 `text/csv` 格式的数据，所以需要转换数据。

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))

In [ ]:
print(response)

可以看出，模型的响应是看起来很复杂的字典，其中包含大量信息。我们最关心的是 `'Body'` 对象，它是一个 streaming 对象，我们需要 `read` Body 以便使用它。

In [ ]:
response = response['Body'].read().decode('utf-8')
print(response)

知道如何处理用户输入数据后，我们可以开始设置基础设施，使网络应用能运行。我们将使用两个不同的服务，分别是 Amazon 的 Lambda 和 API Gateway 服务。

Lambda 服务使用户能够编写相对简单的代码，并且每当所选的触发器被触发时，代码就会执行。例如，每当新数据上传到 S3 上的某个文件夹时，你希望更新数据库。

API Gateway 服务使你能够创建 HTTP 端点（网址），该端点与其他 AWS 服务相连。好处之一是你可以决定要访问这些端点，需要具备哪些凭证（如果需要）。

我们将通过 API Gateway 设置一个可以公开访问的 HTTP 端点。然后，每当有人向该端点发送数据，我们将触发 Lambda 函数，该函数会将输入（影评）发送给模型的端点，然后返回结果。

### 设置 Lambda 函数

首先设置 Lambda 函数。每当公共 API 向其发送数据，该 Lambda 函数就会执行。当它被执行时，它将接收数据，对数据执行必要的处理步骤，将数据（影评）发送给我们创建的 SageMaker 端点，然后返回结果。

#### 第一部分：为 Lambda 函数创建 IAM 角色

因为我们希望 Lambda 函数调用 SageMaker 端点，所以需要确保它有权限这么做。我们将创建一个之后分配给 Lambda 函数的角色。

在 AWS 控制台中转到 **IAM** 页面并点击 **Roles**，然后点击 **Create role**。确保 **AWS service** 属于所选的受信实体类型，并选择 **Lambda** 作为将使用该角色的服务，然后点击 **Next: Permissions**。

在搜索框中输入 `sagemaker` 并选中 **AmazonSageMakerFullAccess** 策略旁边的复选框，然后点击 **Next: Review**。

最后命名此角色。设定一个后面能记住的名称，例如 `LambdaSageMakerRole`。然后点击 **Create role**。

#### 第二部分：创建 Lambda 函数

下面开始真正地创建 Lambda 函数。之前提到，为了处理用户提供的输入并将其发送给端点，我们需要以下两项信息：

 - 端点名称，以及
 - vocabulary 对象。

创建 Lambda 函数后，我们将这些信息复制到 Lambda 函数中。

首先在 AWS 控制台中转到 AWS Lambda 页面并点击 **Create a function**。在下个页面选择 **Author from scratch**。下面命名 Lambda 函数，使用后面能记住的名称，例如 `sentiment_analysis_xgboost_func`。记得选中 **Python 3.6** 运行时，然后选择在上个部分创建的角色。接着点击 **Create Function**。

在下个页面，你将看到关于刚刚创建的 Lambda 函数的一些信息。向下滚动应该会看到一个编辑器，你可以在其中编写一些代码，当 Lambda 函数被触发时，这些代码将执行。使用在上面编写的处理单条影评的代码，并添加到提供的示例 `lambda_handler` 中，代码如下所示。

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

# And we need the regular expression library to do some of the data processing
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words
    
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow


def lambda_handler(event, context):
    
    vocab = "*** ACTUAL VOCABULARY GOES HERE ***"
    
    words = review_to_words(event['body'])
    bow = bow_encoding(words, vocab)

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '***ENDPOINT NAME HERE***',# The name of the endpoint we created
                                       ContentType = 'text/csv',                 # The data format that is expected
                                       Body = ','.join([str(val) for val in bow]).encode('utf-8')) # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')
    
    # Round the result so that our web app only gets '1' or '0' as a response.
    result = round(float(result))

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : str(result)
    }
```

将上述代码复制粘贴到 Lambda 代码编辑器中后，将 `**ENDPOINT NAME HERE**` 部分替换成我们之前部署的端点的名称。你可以使用以下代码单元格获得端点的名称。

In [ ]:
xgb_predictor.endpoint

此外，需要将 vocabulary 字典复制到代码中 `lambda_handler` 方法的开头。以下单元格以易于复制粘贴的格式输出了 vocabulary 字典。

In [ ]:
print(str(vocabulary))

将端点名称添加到 Lambda 函数中后，点击 **Save**。Lambda 函数现在已经在运行。接下来，我们需要让网络应用能够执行 Lambda 函数。

### 设置 API Gateway

Lambda 函数现在已设置好，下面使用 API Gateway 创建新的 API，它将触发我们刚刚创建的 Lambda 函数。

在 AWS 控制台中转到 **Amazon API Gateway**，然后点击 **Get started**。

在下个页面选中 **New API** 并命名新的 API，例如 `sentiment_analysis_web_app`。然后，点击 **Create API**。

我们已经创建了一个 API，但是它目前不能执行任何操作。我们希望它能触发之前创建的 Lambda 函数。

选择 **Actions** 下拉菜单并点击 **Create Method**。新的空方法将创建，打开下拉菜单并选择 **POST**，然后选中它旁边的复选框。

对于交互选项来说，选择 **Lambda Function** 并点击 **Use Lambda Proxy integration**。这个选项会让发送给 API 的数据直接发送给 Lambda 函数，不做任何处理。它还需要返回值必须是正确的响应对象，因为 API Gateway 也不会进行处理。

在 **Lambda Function** 文本框中输入之前创建的 Lambda 函数的名称，然后点击 **Save**。在出现的弹出式方框中点击 **OK**，使 API Gateway 有权调用你创建的 Lambda 函数。

创建 API Gateway 的最后一步是打开 **Actions** 下拉菜单并点击 **Deploy API**。你需要创建新的部署阶段并随意命名，例如 `prod`。

现在已经成功地设置了能访问 SageMaker 模型的公共 API。记得复制或写下调用新创建的公共 API 所需的 URL，因为在下一步将用到该 URL。你可以在页面顶部找到该 URL，它位于文字 **Invoke URL** 旁边，用蓝色标出。

## 第 7 步：部署网络应用

创建了可以公开访问的 API 后，我们可以在网络应用中使用它了。我们提供了简单的静态 HTML 文件，该文件可以使用你刚刚创建的公共 API。

在 `website` 文件夹里有一个文件 `index.html`。请将该文件下载到计算机上，并在任意文本编辑器中打开该文件。请将 **\*\*REPLACE WITH PUBLIC API URL\*\*** 替换成在上一步写下的 URL，然后保存文件。

现在在本地计算机上打开 `index.html`，浏览器将充当本地网络服务器，你可以使用提供的网站与 SageMaker 模型互动。

你还可以将此 HTML 文件托管到任何地方，例如 github，或将静态网站托管到 Amazon 的 S3 上。然后就可以将链接分享给任何人，让他们也能尝试该网站。

> **重要事项**：为了使网络应用能与 SageMaker 端点通信，你必须部署和运行端点。这样的话，就要付费了。当你想要使用网络应用的时候，请运行端点，但是不需要使用的时候，要关闭端点，否则会产生巨额 AWS 费用。

### 删除端点

如果不使用端点，一定要关闭端点。端点按照运行时长计费，所以如果忘记关闭了，可能会产生高昂的费用。

In [ ]:
xgb_predictor.delete_endpoint()

## 可选步骤：清理数据

SageMaker 上的默认 notebook 实例没有太多的可用磁盘空间。当你继续完成和执行 notebook 时，最终会耗尽磁盘空间，导致难以诊断的错误。完全使用完 notebook 后，建议删除创建的文件。你可以从终端或 notebook hub 删除文件。以下单元格中包含了从 notebook 内清理文件的命令。

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir